In [9]:
import mysql.connector

class Importing:
    def __init__(self, Bus_state_name, Table_insert, table_name, database_name):
        self.Bus_state_name = Bus_state_name
        self.Table_insert = Table_insert
        self.table_name = table_name
        self.database_name = database_name

        # Connect to MySQL
        self.mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            autocommit=True  # Enable autocommit
        )

        print(self.mydb)
        self.mycursor = self.mydb.cursor(buffered=True)

        # Create the database
        self.mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.database_name}")
        self.mycursor.execute(f"USE {self.database_name}")  # Switch to the new database

        # Escape table names with backticks to handle spaces and reserved keywords
        table_name_1 = f"`{self.table_name}`"

        # Create the table with the specified schema, including Bus_route_name
        self.mycursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name_1} (
                id INT NOT NULL AUTO_INCREMENT,
                Bus_state_name VARCHAR(100),
                Bus_route_name VARCHAR(100),
                Bus_Operator_type VARCHAR(15),
                BusName VARCHAR(50),
                BusType VARCHAR(50),
                Departing_Time TIME,
                Duration VARCHAR(30),
                Reaching_Time TIME,
                Star_rating FLOAT(5),
                Price FLOAT(10),
                Seats_available INT(5),
                PRIMARY KEY (id)
            )
        """)

        # Insert data into the table
        for bus_route_name, operator_data in self.Table_insert.items():
            for operator_type in ['Private', 'Government']:
                print(f"Inserting data for operator type: {operator_type} on route: {bus_route_name}")
                try:
                    bus_name = operator_data[operator_type]['Bus_Name']
                    bus_type = operator_data[operator_type]['Bus_Type']
                    departing_time = operator_data[operator_type]['Departing_Time']
                    duration = operator_data[operator_type]['Duration']
                    reaching_time = operator_data[operator_type]['Reaching_Time']
                    star_rating = operator_data[operator_type]['Star_Rating']
                    price = operator_data[operator_type]['Price']
                    seats_available = operator_data[operator_type]['Seat_availability']
                except KeyError as e:
                    print(f"KeyError: {e} in route {bus_route_name} for operator type {operator_type}. Skipping...")
                    continue

                insert_query = f"INSERT INTO {table_name_1} (Bus_state_name, Bus_route_name, Bus_Operator_type, BusName, BusType, Departing_Time, Duration, Reaching_Time, Star_rating, Price, Seats_available) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"

                # Get the minimum length among the lists to avoid index errors
                value_lengths = [len(i) for i in [bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seats_available]]
                min_length = min(value_lengths)
                print(f"Minimum length of lists: {min_length}")

                # Insert each value in the list as a separate row
                for i in range(min_length):
                    print(f"Inserting row {i+1} for {bus_route_name} - {operator_type}")
                    print((
                        self.Bus_state_name,
                        bus_route_name,
                        operator_type, 
                        bus_name[i], 
                        bus_type[i], 
                        departing_time[i], 
                        duration[i], 
                        reaching_time[i], 
                        star_rating[i], 
                        price[i], 
                        seats_available[i]
                    ))
                    self.mycursor.execute(insert_query, (
                        self.Bus_state_name,
                        bus_route_name,
                        operator_type, 
                        bus_name[i], 
                        bus_type[i], 
                        departing_time[i], 
                        duration[i], 
                        reaching_time[i], 
                        star_rating[i], 
                        price[i], 
                        seats_available[i]
                    ))
                    self.mydb.commit()  # Commit after each insertion




In [12]:
import pandas as pd

# Assuming 'TSRTC_data' is a dictionary containing the scraped data
TSRTC_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/TSRTC.json") # Change read_csv to read_json

#Inputing the arguments State bus name,Variable containing scraped data,Table name,Database name
importer = Importing('TSRTC-Telangana State Transport', TSRTC_data, "Bus_Data", "RedBus")


Inserting data for operator type: Private on route: Khammam to Hyderabad
Minimum length of lists: 25
Inserting row 1 for Khammam to Hyderabad - Private
('TSRTC-Telangana State Transport', 'Khammam to Hyderabad', 'Private', 'Sri KVR Travels', 'A/C Sleeper (2+1)', '23:20', '05h 40m', '05:00', 3.2, '560', 19)
Inserting row 2 for Khammam to Hyderabad - Private
('TSRTC-Telangana State Transport', 'Khammam to Hyderabad', 'Private', 'Sri KVR Travels', 'Non A/C Seater / Sleeper (2+1)', '23:40', '05h 20m', '05:00', 3.4, '400', 25)
Inserting row 3 for Khammam to Hyderabad - Private
('TSRTC-Telangana State Transport', 'Khammam to Hyderabad', 'Private', 'BSR Tours And Travels', 'NON A/C Seater (2+2)', '23:40', '06h 20m', '06:00', 4.2, '400', 31)
Inserting row 4 for Khammam to Hyderabad - Private
('TSRTC-Telangana State Transport', 'Khammam to Hyderabad', 'Private', 'Bharathi Tours & Travels', 'Non A/C Seater / Sleeper (2+1)', '23:40', '04h 20m', '04:00', 3.8, '400', 27)
Inserting row 5 for Khammam

In [14]:
import pandas as pd

# Assuming 'TSRTC_data' is a dictionary containing the scraped data
assam_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/ASTC.json") # Change read_csv to read_json
andhar_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/APSRTC.json")
haryana_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/HRTC.json")
uttar_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/UPSRTC.json")
south_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/SBSTC.json")
kerala_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/Kerala.json")
jammu_data =pd.read_json("C:/Users/yuvanelson/Documents/Rebbus-Project-1-main/Rebbus-Project-1-main/JKSRTC.json")

In [15]:
#Inputing the arguments State bus name,Variable containing scraped data,Table name,Database name
importer = Importing('ASTC-Assam State Transport', assam_data, "Bus_Data", "RedBus")
importer = Importing('APSRTC-Andhra Pradesh State Transport', andhar_data, "Bus_Data", "RedBus")
importer = Importing('HRTC-Haryana State Transport', haryana_data, "Bus_Data", "RedBus")
importer = Importing('UPSRTC-Uttar Pradesh State Transport', uttar_data, "Bus_Data", "RedBus")
importer = Importing('SBSTC-South Bengal State Transport', south_data, "Bus_Data", "RedBus")
importer = Importing('kerala-kerala State Transport', kerala_data, "Bus_Data", "RedBus")
importer = Importing('JKSRTC-Jammu and Kashmir State Road Transport Corporation', jammu_data, "Bus_Data", "RedBus")



Inserting data for operator type: Private on route: Tezpur to Guwahati
Minimum length of lists: 3
Inserting row 1 for Tezpur to Guwahati - Private
('ASTC-Assam State Transport', 'Tezpur to Guwahati', 'Private', 'Assam State Transport Corporation (ASTC) - 157959', 'Bharat Benz A/C Seater (2+2)', '14:00', '04h 30m', '18:30', 4.3, '298', 12)
Inserting row 2 for Tezpur to Guwahati - Private
('ASTC-Assam State Transport', 'Tezpur to Guwahati', 'Private', 'Orient Transline', 'A/C Seater (2+1)', '07:35', '03h 25m', '11:00', 3.7, '342.86', 18)
Inserting row 3 for Tezpur to Guwahati - Private
('ASTC-Assam State Transport', 'Tezpur to Guwahati', 'Private', 'WARISPIYA TRAVELS', 'AC Seater (2+2)', '07:45', '04h 15m', '12:00', 4.2, '360', 35)
Inserting data for operator type: Government on route: Tezpur to Guwahati
KeyError: 'Bus_Name' in route Tezpur to Guwahati for operator type Government. Skipping...
Inserting data for operator type: Private on route: Guwahati to Tezpur
Minimum length of lists: